In [1]:
import os
import copy
import csv
import numpy as np
import pandas as pd
import brainbox.io.one as bbone
import ibllib.atlas as atlas
import pickle
from one.api import ONE
one = ONE(base_url='https://alyx.internationalbrainlab.org')

C:\Users\Dan\.conda\envs\iblenv\lib\site-packages\one\api.py:1294: UserWarning: Newer cache tables require ONE version 1.10.0 or greater
  warnings.warn(f'Newer cache tables require ONE version {min_version} or greater')


In [2]:
def get_bwm_sessions():

    '''
    Gaelle, 06.04.2022
    '''

    str_query = (  
       'session__project__name__icontains,ibl_neuropixel_brainwide_01,'
       'session__json__IS_MOCK,False,session__qc__lt'
       ',50,~json__qc,CRITICAL,'
       'session__extended_qc__behavior,1,'
       'json__extended_qc__tracing_exists,True,'
       '~session__extended_qc___task_stimOn_goCue_delays__lt,0.9,'
       '~session__extended_qc___task_response_feedback_delays__lt,0.9,'
       '~session__extended_qc___task_wheel_move_before_feedback__lt,0.9,'
       '~session__extended_qc___task_wheel_freeze_during_quiescence__lt,0.9,'
       '~session__extended_qc___task_error_trial_event_sequence__lt,0.9,'
       '~session__extended_qc___task_correct_trial_event_sequence__lt,0.9,'
       '~session__extended_qc___task_reward_volumes__lt,0.9,'
       '~session__extended_qc___task_reward_volume_set__lt,0.9,'
       '~session__extended_qc___task_stimulus_move_before_goCue__lt,0.9,'
       '~session__extended_qc___task_audio_pre_trial__lt,0.9')                  
        
    str_query2 = (
       'session__project__name__icontains,ibl_neuropixel_brainwide_01,'
       'session__json__IS_MOCK,False,session__qc__lt,50,'
       '~json__qc,CRITICAL,session__extended_qc__behavior,1,'       
       'json__extended_qc__tracing_exists,True,'
       'session__extended_qc___experimenter_task,PASS')        
            
    ins = np.concatenate([one.alyx.rest('insertions', 'list', django = x)
                          for x in [str_query, str_query2]])

    eid_probe = set([x['session']+'_'+x['name'] for x in ins]) # pid via x['id']
    ins = [x.split('_') for x in eid_probe] 
 
    return ins

In [4]:
ins = get_bwm_sessions()

In [8]:
with open("bwm_eid.pkl",'wb') as f:
    pickle.dump(ins, f)

In [12]:
from brainbox.io.one import SpikeSortingLoader


In [ ]:
eids = np.load("data/brainwide_map_eids_ephys_histology.npy")


In [ ]:
# Load the UUID index file
with open(UUID_TO_INDEX_PATH, 'rb') as uuid_file:
  uuids = pickle.load(uuid_file)

In [ ]:
sess_info = one.alyx.rest("sessions","list")
eid2lab = {}
for sess in sess_info:
    eid = one.to_eid(sess['url'])
    eid2lab[eid] = sess['lab']

In [16]:
# build a CSV file which tracks the UUID and lab that each cluster came from
with open('data/metadata.csv','w', newline='') as csvfile:
  writer = csv.writer(csvfile)
  for uuid, index in uuids.items():
    eid = data['eid']
    writer.writerow([uuid, eid2lab[eid]])

TypeError: 'int' object is not subscriptable

In [13]:
from matplotlib import cm
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [20, 10]
import time

In [17]:
# Load all of the ISI_AMP csv files and populate a big matrix with the cluster count data
# then plot slices of this to look at the cluster count
isi_amp_dir = 'C:\\proj\VBL\\nptraj-ephys-server\\pipeline\\isi_amp_data\\'

files = os.listdir(isi_amp_dir)

# 528*320*456  ap/dv/lr
cluCount = np.zeros((456,528,320))

for file in files:
    df = pd.read_csv(isi_amp_dir + file)
    df.reset_index()
    for i,row in df.iterrows():
        cluCount[int(row['ml']/40),int(row['ap']/40),int(row['dv']/40)] = row['clu_count']

In [209]:
# New plan:
# Load each ISI_AMP file and save each leaf node area separately as a CSV
# Average the ISI distribution for that area (or do PCA on it?)

isi_amp_dir = 'C:\proj\IBL\cluster_pipelines\data\isi_amp_data'
files = os.listdir(isi_amp_dir)

df = pd.read_csv('C:\proj\IBL\cluster_pipelines\data\isi_amp_data\\' + files[0])
# dictionary containing area data
areaData = pd.DataFrame(columns=['area',df.columns[9:].values])
i = 0

for file in files:
    df = pd.read_csv('C:\proj\IBL\cluster_pipelines\data\isi_amp_data\\' + file)
    df.reset_index()
    for i,row in df.iterrows():
        area = row['atlas_id']
        data = row[9:].values
        areaData.loc[i] = [area, data]
        i += 1

AttributeError: 'numpy.float64' object has no attribute 'values'

In [203]:
for area in areaDict.keys():
    areaData = areaDict[area]
    areaMean = np.average(areaData,axis=0)
    #np.savetxt('C:\proj\IBL\cluster_pipelines\data\isi_amp_areas\\' + str(area) + '.csv',areaData,delimiter=',')

In [216]:
np.average(areaDict[0],axis=0)

array([0.02462212, 0.04363561, 0.10895899, 0.15841355, 0.15662258,
       0.15228122, 0.12637609, 0.10094896, 0.0703425 , 0.05779838])

In [213]:
areaMean = np.average(areaData,axis=0)


C:\Users\Dan\.conda\envs\iblenv\lib\site-packages\numpy\lib\function_base.py:495: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)


ZeroDivisionError: division by zero